## Bonus – Extract Vector Query from a prompt

In [ ]:
!pip install cohere

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
AI_STUDIO_KEY = os.getenv("GOOGLE_AI_STUDIO_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

if(COHERE_API_KEY == None):
    print("COHERE_API_KEY is missing. You can't complete this exercise without it.")
    print("Go to https://dashboard.cohere.com/api-keys to get your key")

## Generate Query from Prompt

In [ ]:
import cohere, os
co = cohere.Client(COHERE_API_KEY)

def generate_query_from_promt(prompt):
    response = co.chat(
        message=prompt,
        search_queries_only=True
    )
    
    return response.search_queries[0]["text"]

### Test

In [ ]:
# Example of how to generate a query from a prompt
generate_query_from_promt("Where do the tallest penguins live?")

In [ ]:
generate_query_from_promt("What can you tell me about zero shot training? Please answer based on the provided content")

## Two-step RAG with generated prompt
### Connect

In [ ]:
import weaviate

# Connect to the local instance deployed with Docker Compose
client = weaviate.connect_to_local(
    headers={
       "X-Google-Api-Key": AI_STUDIO_KEY,
    },
    
    # Set longer timeout
    additional_config=weaviate.config.AdditionalConfig(timeout=(500, 1500))
)

client.is_ready()

In [ ]:
# import weaviate, os

# client = weaviate.connect_to_wcs(
#     cluster_url=os.getenv("WORKSHOP_DEMO_URL"),
#     auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WORKSHOP_DEMO_KEY_READ")),

#     headers={
#        "X-Palm-Api-Key": AI_STUDIO_KEY,
#     }

#     # Set longer timeout
#     additional_config=weaviate.config.AdditionalConfig(timeout=(500, 1500))
# )

# client.is_ready()

### Query

In [ ]:
from weaviate.classes.query import Filter

def two_step_rag(user_prompt):
    # Step 1
    prompt = user_prompt + " Please only use the provided content with this prompt. Don't make things up."
    
    generated_query = generate_query_from_promt(prompt)
    print("=== Generated Query ===")
    print(f"Generated query: {generated_query}")

    # Step 2
    papers = client.collections.get("Papers")

    response = papers.generate.near_text(
        query=generated_query,
        limit=5,
        grouped_task=prompt,
        grouped_properties=["chunk", "title"]
    )

    # Print results
    print("=== Generated Response ===")
    print(response.generated)

    print("=== Source ===")
    for item in response.objects:
        print(item.properties["title"])
        print(item.properties["chunk"])
        print("——————————————————————————————")

In [ ]:
two_step_rag("What can you tell me about zero shot training?")

In [ ]:
two_step_rag("What are the challenges of zero shot training?")

In [ ]:
client.close()